In [411]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import brentq
from datetime import time
import warnings
warnings.filterwarnings('ignore')

In [161]:
#Global Variables

risk_free_rate=0.05
timeframe= '30min'
trading_sessions= 14
lookback= 10

In [99]:
#Math Logic

# Black-Scholes formula
def black_scholes_price(S, K, T, r, sigma, option_type="C"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "C":
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "P":
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

# Implied Volatility using Newton Raphson
def calculate_iv(option_price, S, K, T, r, option_type="C"):
    def bs_error(sigma):
        return black_scholes_price(S, K, T, r, sigma, option_type) - option_price

    try:
        return brentq(bs_error, 1e-6, 3)  
    except ValueError:
        return np.nan  

#Find z-score for mean reversion
def calculate_zscore(data, lookback):
    mean = data.rolling(window=lookback).mean()
    std = data.rolling(window=lookback).std()
    zscore = (data - mean) / std
    return zscore

#Calculates sharpe for a pnl series
def calculate_sharpe_ratio(returns, risk_free_rate=0):
    excess_returns = returns - risk_free_rate
    sharpe_ratio= excess_returns.mean() / excess_returns.std()
    return sharpe_ratio

#Max drawdown of a pnl series
def calculate_drawdown(returns):
  cumulative_returns = (1 + returns).cumprod()
  peak = cumulative_returns.cummax()
  drawdown = (cumulative_returns - peak) / peak
  return drawdown

In [685]:
# Data operations

#Load Data
def load_data(file_path):
    data = pd.read_csv(file_path, parse_dates=['start_ts'], usecols=['symbol','start_ts','open','high','low','close','volume','vwap'])
    data = data.dropna()
    return data

def strip_extra_trading_hours(data):
    
    data=data.reset_index()

    data = data.drop(data[data['start_ts'].dt.time< time(9,30,0)].index)
    data = data.drop(data[data['start_ts'].dt.time>= time(16,0,0)].index)

    return data

'''Filter the Data
    1. Convert to 30min timeframe
    2. Strip off extra trading hours
'''
def convert_eqt_data(eqt,new_timeframe):  

    eqt = eqt.set_index('start_ts')

    eqt_data = eqt.resample(new_timeframe).agg(    
        close= ('close', 'last'),
        Weighed_Price_Volume= ('vwap', lambda x:(x* eqt.loc[x.index, 'volume']).sum()),
        volume= ('volume','sum'),
        symbol= ('symbol','last')
    )

    eqt_data['vwap'] = eqt_data['Weighed_Price_Volume']/eqt_data['volume']
    eqt_data=eqt_data.drop(columns=['Weighed_Price_Volume'])

    #Strip off extra trading hours
    eqt_data=strip_extra_trading_hours(eqt_data)

    eqt_data= eqt_data.reset_index()
    eqt_data = eqt_data.dropna()
    return eqt_data

#As seen the data is alredy sorted
def data_filtering(option):

    #Add columns 
    option_characteristics=pd.DataFrame((x.split('_') for x in option['symbol']), columns=['ticker','expiry_date','strike_price','option_type'])
    option_characteristics['expiry_date']=pd.to_datetime(option_characteristics['expiry_date'])
    option_characteristics['strike_price']= option_characteristics['strike_price'].astype(float)
    option=pd.concat([option,option_characteristics],axis=1)

    option=option.drop(option[option['option_type']=='P'].index)

    option=option.reset_index()

    return option


'''Computationally cumbersome- Assumptions
        1. Selected the strike just greater than spot may not be closest but should be fairly liquid
        2. TTE != 0DTE
        3. Trading only on option closes
'''
def convert_option_data(option,new_timeframe):  

    option_close= option.pivot(index='start_ts', columns='symbol', values='close')
    option_close= option_close.resample(new_timeframe).last()
    option_close= strip_extra_trading_hours(option_close)    

    # option_volume= option.pivot(index='start_ts', columns='symbol', values='volume')
    # option_volume= option_volume.resample(new_timeframe).sum()
    # option_volume= strip_extra_trading_hours(option_volume)         

    return option_close

def precompute_spread(eqt, option_close, option_all):

    spread=pd.DataFrame()

    option_close=option_close.set_index('start_ts')

    for _,rows in eqt.iterrows():
        symbol=rows['symbol']
        start_ts=rows['start_ts']
        spot_price=rows['close']
        
        #This ensures we have no 0 DTE!
        valid_expires = option_all[option_all['expiry_date'] > rows['start_ts']]
        closest_expires= valid_expires['expiry_date'].unique()[:2]
        near_expiry= str(closest_expires[0].date().strftime("%Y%m%d"))
        far_expiry= str(closest_expires[1].date().strftime("%Y%m%d"))

        otm_strikes= option_all[option_all['strike_price'] >= spot_price]
        nearest_atm_strike= int(otm_strikes.iloc[0]['strike_price'])

        option1=f"{symbol}_{near_expiry}_{nearest_atm_strike}_C"
        option2=f"{symbol}_{far_expiry}_{nearest_atm_strike}_C"

        option1_price= option_close.loc[start_ts,option1]
        option2_price= option_close.loc[start_ts,option2]

        # print(start_ts, option1, option1_price, option2, option2_price)
        
        tte1=(closest_expires[0]- start_ts)/pd.Timedelta(days=365)
        tte2=(closest_expires[1]- start_ts)/pd.Timedelta(days=365)

        iv1=calculate_iv(option1_price, spot_price, nearest_atm_strike, tte1 ,risk_free_rate, 'C')
        iv2=calculate_iv(option2_price, spot_price, nearest_atm_strike, tte2 ,risk_free_rate, 'C')

        temp={}
        temp['start_ts']= start_ts
        temp['option']= option2_price -option1_price
        temp['iv']= iv2 - iv1
        temp['symbol']= f"{option1}*{option2}"
        temp['expiry']=pd.to_datetime(closest_expires[0].strftime('%Y-%m-%d 15:30:00'))
        temp['delta']=0.0
        temp['spot']=spot_price
        
        spread=pd.concat([spread, pd.DataFrame([temp])])

    return spread

def create_lag(option_close_eod):

    option_close_eod_diff=option_close_eod.diff()
    option_close_eod_diff=option_close_eod_diff.reset_index()

    #Ensure format
    option_close_eod_diff['start_ts_shifted']= option_close_eod_diff['start_ts'].shift(1)
    option_close_eod_diff = option_close_eod_diff.dropna(subset=['start_ts_shifted'])
    option_close_eod_diff['start_ts_shifted'] = pd.to_datetime(option_close_eod_diff['start_ts_shifted'])
    option_close_eod_diff=option_close_eod_diff.set_index('start_ts_shifted')

    # Add a new row with NaN values
    option_close_eod_diff.loc[option_close_eod_diff.index[-1] + pd.Timedelta(days=1)] = [np.nan] * (len(option_close_eod_diff.columns))

    return option_close_eod_diff
    

In [220]:
# Feature Engineering
def generate_features(spread, window=10):

    spread['iv_zscore']=calculate_zscore(spread['iv'],window)
    spread['price_zscore']=calculate_zscore(spread['option'],window)
    
    # Lagged features
    # for lag in range(1, 4):
    #     spread[f'iv_lag{lag}'] = spread['iv'].shift(lag)
    #     spread[f'price_lag{lag}'] = spread['price'].shift(lag)
    #     spread[f'iv_zscore{lag}'] = spread['iv_zscore'].shift(lag)
    
    # # Target: Binary classification (Profit/Loss based on future spread return)
    # spread['pnl'] = (spread['price'].shift(-1) - spread['price']) > 0
    # spread['pnl'] = spread['pnl'].astype(int)
    
    return spread.dropna()

In [748]:
def generate_signals(spread , option_close, lower_threshold=-1, upper_threshold=1):

    #Data lags behind 1 week
    last_trade_date=spread.iloc[-1]['start_ts']

    option_close_eod=option_close[option_close['start_ts'].dt.time == pd.to_datetime('15:30:00').time()]
    option_close_eod= option_close_eod.set_index('start_ts')
    # print(option_close_eod.to_string())

    option_close_eod_diff=create_lag(option_close_eod)
    # print(option_close_eod_diff.to_string())
    
    #Pnl 
    daily_pnl= pd.DataFrame({'start_ts': spread['start_ts'], 'pnl':0.0})
    daily_pnl=daily_pnl[daily_pnl['start_ts'].dt.time == pd.to_datetime('15:30:00').time()]
    daily_pnl.set_index('start_ts',inplace=True)
    daily_pnl.loc[daily_pnl.index[-1] + pd.Timedelta(days=1)] = [np.nan] * (len(daily_pnl.columns))
    # position= pd.DataFrame(0.0, index=spread['start_ts'],columns=spread['symbol'].unique())

    for _,rows in spread.iterrows():
        start_ts= rows['start_ts']
        eod_label= start_ts.replace(hour=15, minute=30, second=0)
        symbol= rows['symbol']
        contracts= symbol.split('*')
        eod_price= option_close_eod.loc[eod_label,contracts[1]]-option_close_eod.loc[eod_label,contracts[0]]
        expiry= rows['expiry']
        zscore= rows['price_zscore']
        price= rows['option']

        # print(start_ts,symbol,eod_price,expiry,zscore,price)

        if expiry > last_trade_date: #No expiry Trades
            break

        if zscore < lower_threshold: #Long trade
            # print("Long",start_ts,symbol,eod_price-price,expiry,zscore)
            daily_pnl.loc[eod_label,'pnl']+= eod_price - price #Today Pnl

            #Fwd Pnl
            temp_pnl = pd.DataFrame(0.0,index=daily_pnl.index, columns=['pnl'])
            temp_pnl.loc[eod_label:expiry,'pnl']= option_close_eod_diff.loc[eod_label:expiry,contracts[1]]-option_close_eod_diff.loc[eod_label:expiry,contracts[0]]
            temp_pnl['pnl']=temp_pnl['pnl'].shift(1) 
            temp_pnl['pnl']=temp_pnl['pnl'].fillna(0.0)
            daily_pnl['pnl']+=temp_pnl['pnl']

            # print(temp_pnl.to_string(),daily_pnl.to_string())
        elif zscore > upper_threshold: #Short trade
            print("Short",start_ts,symbol,eod_price-price,expiry,zscore)
            daily_pnl.loc[eod_label,'pnl']+= -(eod_price - price) #Today Pnl

            #Fwd Pnl
            temp_pnl = pd.DataFrame(0.0,index=daily_pnl.index, columns=['pnl'])
            temp_pnl.loc[eod_label:expiry,'pnl']= -(option_close_eod_diff.loc[eod_label:expiry,contracts[1]]-option_close_eod_diff.loc[eod_label:expiry,contracts[0]])
            temp_pnl['pnl']=temp_pnl['pnl'].shift(1) 
            temp_pnl['pnl']=temp_pnl['pnl'].fillna(0.0)
            daily_pnl['pnl']+=temp_pnl['pnl']

            # print(temp_pnl.to_string(),daily_pnl.to_string())
        
    # cumulative_pnl = pnl.cumsum()
    # sharpe = calculate_sharpe_ratio(pnl)
    # drawdown = calculate_drawdown(pnl)
    # max_drawdown = drawdown.min()  
    
    # print(f"Sharpe Ratio: {sharpe}")
    # print(f"Maximum Drawdown: {max_drawdown}")
    return daily_pnl

In [460]:
'''Equity trading hours from 4am to 8pm'''
eqt_path = 'CBOE Data/testeqt.csv'
eqt = load_data(eqt_path)

eqt_data = convert_eqt_data(eqt,timeframe)
print(eqt_data.to_string())

    index            start_ts     close    volume symbol        vwap
0      11 2023-01-03 09:30:00  126.9336  13683695   AAPL  129.271024
1      12 2023-01-03 10:00:00  125.4600  10686686   AAPL  126.092497
2      13 2023-01-03 10:30:00  125.5950   9023652   AAPL  125.404657
3      14 2023-01-03 11:00:00  125.3400   8429008   AAPL  125.209053
4      15 2023-01-03 11:30:00  124.7488   6110185   AAPL  124.879579
5      16 2023-01-03 12:00:00  124.6554   4796729   AAPL  124.763794
6      17 2023-01-03 12:30:00  124.5900   4015050   AAPL  124.626831
7      18 2023-01-03 13:00:00  124.6500   4074701   AAPL  124.427184
8      19 2023-01-03 13:30:00  124.8150   4116170   AAPL  124.566492
9      20 2023-01-03 14:00:00  124.5726   3589739   AAPL  124.678465
10     21 2023-01-03 14:30:00  124.6150   5941235   AAPL  124.697510
11     22 2023-01-03 15:00:00  124.6200   5301049   AAPL  124.698999
12     23 2023-01-03 15:30:00  125.0200  10287122   AAPL  125.052305
13     59 2023-01-04 09:30:00  126

In [602]:
'''Options trading hours from 9.30am to 4pm = 14 Trading sessions of 0.5hr'''
option_path = 'CBOE Data/testopt.csv'
option = load_data(option_path)
option_all= data_filtering(option)

#Convert to 30min Timeframe
option_close = convert_option_data(option_all,timeframe)
# print(option_close.to_string())

spread= precompute_spread(eqt_data,option_close,option_all)
spread= generate_features(spread,10)

print(spread.to_string())

             start_ts  option        iv                                   symbol              expiry  delta      spot  iv_zscore  price_zscore
0 2023-01-03 14:00:00    1.44 -0.131022  AAPL_20230106_125_C*AAPL_20230113_125_C 2023-01-06 15:30:00    0.0  124.5726  -1.017077     -0.449719
0 2023-01-03 14:30:00    1.47 -0.129440  AAPL_20230106_125_C*AAPL_20230113_125_C 2023-01-06 15:30:00    0.0  124.6150  -0.700951      0.201864
0 2023-01-03 15:00:00    1.43 -0.128920  AAPL_20230106_125_C*AAPL_20230113_125_C 2023-01-06 15:30:00    0.0  124.6200  -1.203330     -1.524503
0 2023-01-03 15:30:00    1.50 -0.114424  AAPL_20230106_126_C*AAPL_20230113_126_C 2023-01-06 15:30:00    0.0  125.0200   1.060471      0.987421
0 2023-01-04 09:30:00    1.50 -0.160269  AAPL_20230106_127_C*AAPL_20230113_127_C 2023-01-06 15:30:00    0.0  126.6788  -2.548394      0.807294
0 2023-01-04 10:00:00    1.46 -0.168187  AAPL_20230106_126_C*AAPL_20230113_126_C 2023-01-06 15:30:00    0.0  125.9912  -2.004442     -0.607644

In [746]:

results=generate_signals(spread, option_close)
# print(results.to_string())
    

2023-01-03 15:00:00 AAPL_20230106_125_C*AAPL_20230113_125_C 0.06999999999999984 2023-01-06 15:30:00 -1.5245027578668433
                      pnl
start_ts                 
2023-01-03 15:30:00  0.00
2023-01-04 15:30:00 -0.09
2023-01-05 15:30:00  0.36
2023-01-06 15:30:00 -0.91
2023-01-07 15:30:00  0.00                       pnl
start_ts                 
2023-01-03 15:30:00  0.07
2023-01-04 15:30:00 -0.09
2023-01-05 15:30:00  0.36
2023-01-06 15:30:00 -0.91
2023-01-07 15:30:00   NaN
2023-01-04 10:30:00 AAPL_20230106_127_C*AAPL_20230113_127_C 0.18999999999999995 2023-01-06 15:30:00 -2.055480479109446
                      pnl
start_ts                 
2023-01-03 15:30:00  0.00
2023-01-04 15:30:00  0.00
2023-01-05 15:30:00  0.01
2023-01-06 15:30:00 -0.14
2023-01-07 15:30:00  0.00                       pnl
start_ts                 
2023-01-03 15:30:00  0.07
2023-01-04 15:30:00  0.10
2023-01-05 15:30:00  0.37
2023-01-06 15:30:00 -1.05
2023-01-07 15:30:00   NaN
